In [ ]:
# -------------------------------------------------------------------------------#
#                        Changes to the minihack RewardManager                   #
# -------------------------------------------------------------------------------#

# The following code has been copied from the reward_manager file from the minihack repository.
# For ease of demonstration, all parts that have not been changed were excluded from this file.

# The base 'Event' class was adapted so that an input of the neighbouring cells one step before, are
# included. Here, instead of it being assumed the agent is standing on top of an item, even when the
# item does not exist in the environment, the object must first be present in a neighbouring cell.
class Event(ABC):
    # All other class info has been excluded for brevity.

    def _standing_on_top(env, location, past_cells=[]):
        # CHANGED HERE -- and all calls to check include the past_cells too
        return location in past_cells and not env.screen_contains(location)


class RewardManager(AbstractRewardManager):
    # CHANGED HERE -- the addition of past_cells is included in the checks for all events added
    # to the reward manager. Thus, all other event types had this added as a parameter.
    def check_episode_end_call(
        self, env, previous_observation, action, observation
    ) -> bool:

        reward = 0.0
        for event in self.events:
            if event.achieved:
                continue
            reward += event.check(
                # ADDED HERE
                env, previous_observation, action, observation, self.past_cells
            )

        for custom_reward_function in self.custom_reward_functions:
            reward += custom_reward_function(
                env, previous_observation, action, observation
            )
            
        self.past_cells = env.get_neighbor_descriptions()
        self._reward = reward
        return self._check_complete()

In [ ]:
# -------------------------------------------------------------------------------#
#                                    New Event Type                              #
# -------------------------------------------------------------------------------#
class InventoryEvent(Event):
    """An event which checks whether a specified object is in the agent's inventory."""

    def __init__(self, *args, inv_item: str):
        super().__init__(*args)
        """Initialise the Event.

        Args:
            inv_item (str):
                The name of the object to gain.
            reward (float):
                The reward for the event occuring
            repeatable (bool):
                Whether the event can occur repeated (i.e. if the reward can be
                collected repeatedly
            terminal_required (bool):
                Whether this event is required for the episode to terminate.
            terminal_sufficient (bool):
                Whether this event causes the episode to terminate on its own.
        """
        self.inv_item = inv_item

    def check(self, env, previous_observation, action, observation) -> float:
        inventory_items = observation[env._original_observation_keys.index("inv_strs")]
        for inv_item in inventory_items:
          if self.inv_item in inv_item[: np.where(inv_item == 0)[0][0]].tobytes().decode("utf-8"):
            return self._set_achieved()
        return 0.0